In [26]:
import pandas as pd
from PyPDF2 import PdfReader as PDFReader, PdfWriter as PDFWriter
import os

In [27]:
os.listdir('./')

['.ipynb_checkpoints', 'BRGY TUGATOG.pdf', 'PDFS', 'test.ipynb']

In [46]:
params = {
    'number': {
        'upper_left': (0, 630),
        'lower_right': (60, 80),
        'upper_right': (60, 630),
        'lower_left': (0, 80),
    },
    'legend_and_voters_name': {
        'upper_left': (60, 630),
        'lower_right': (360, 80),
        'upper_right': (360, 630),
        'lower_left': (60, 80),
    },
    'address': {
        'upper_left': (360, 630),
        'lower_right': (580, 80),
        'upper_right': (580, 630),
        'lower_left': (360, 80),
    }
}

end_line = "We CERTIFY that the above voting records are true, correct and complete."

In [ ]:
# checks if the current page contains the end of the list for a certain precinct number
cur = PDFReader('BRGY TUGATOG.pdf')
list_ends = []
for i in range(len(cur.pages)):
    # print(cur.pages[31].extract_text())
    lst = cur.pages[i].extract_text().split('\n')
    # number = 
    # print(lst[8].split(' ')[0])
    check_end_line = False
    for ind in range(len(lst)):
        if lst[ind] == end_line:
            check_end_line = True
            list_ends.append(i)
            break

In [55]:
for pdfs in os.listdir('./PDFS/'):
    cur = PDFReader(f'./PDFS/{pdfs}')
    output = PDFWriter()
    for i in range(len(cur.pages)):
        for key, index in enumerate(params):
            pg = cur.pages[i]
            ind = params[index]
            pg.cropbox.upper_left = (ind['upper_left'][0], ind['upper_left'][1])
            pg.cropbox.lower_right = (ind['lower_right'][0], ind['lower_right'][1])
            pg.cropbox.upper_right = (ind['upper_right'][0], ind['upper_right'][1])
            pg.cropbox.lower_left = (ind['lower_left'][0], ind['lower_left'][1])
            output.add_page(pg)
    with open(f'./Created PDFs/{pdfs}.pdf', "wb") as out_f:
        output.write(out_f)
    break